In [ ]:
%matplotlib inline

# Scalable diagonal discipline.

Let us consider the
[SobieskiAerodynamics][gemseo.problems.mdo.sobieski.disciplines.SobieskiAerodynamics] discipline.
We want to build its [DataDrivenScalableDiscipline][gemseo.problems.mdo.scalable.data_driven.discipline.DataDrivenScalableDiscipline] counterpart,
using a [ScalableDiagonalModel][gemseo.problems.mdo.scalable.data_driven.diagonal.ScalableDiagonalModel]

For that, we can use a 20-length [DiagonalDOE][gemseo.algos.doe.diagonal_doe.diagonal_doe.DiagonalDOE]
and test different sizes of variables or different settings
for the scalable diagonal discipline.


In [ ]:
from __future__ import annotations

from gemseo import create_discipline
from gemseo import create_scalable
from gemseo import create_scenario
from gemseo.problems.mdo.sobieski.core.design_space import SobieskiDesignSpace

## Training dataset

The first step is to build an [BaseFullCache][gemseo.caches.base_full_cache.BaseFullCache] dataset
from a [DiagonalDOE][gemseo.algos.doe.diagonal_doe.diagonal_doe.DiagonalDOE].



### Instantiate the discipline

For that, we instantiate the
[SobieskiAerodynamics][gemseo.problems.mdo.sobieski.disciplines.SobieskiAerodynamics] discipline
and set it up to cache all evaluations.



In [ ]:
discipline = create_discipline("SobieskiAerodynamics")

### Get the input space

We also define the input space on which to sample the discipline.



In [ ]:
input_space = SobieskiDesignSpace()
input_names = [name for name in discipline.io.input_grammar.names if name != "c_4"]
input_space.filter(input_names)

### Build the DOE scenario

Lastly, we sample the discipline by means of a [DOEScenario][gemseo.scenarios.doe_scenario.DOEScenario]
relying on both discipline and input space.
In order to build a diagonal scalable discipline,
a [DiagonalDOE][gemseo.algos.doe.diagonal_doe.diagonal_doe.DiagonalDOE] must be used.



In [ ]:
scenario = create_scenario(
    [discipline],
    "y_2",
    input_space,
    scenario_type="DOE",
    formulation_name="DisciplinaryOpt",
)
for output_name in discipline.io.output_grammar.names:
    if output_name != "y_2":
        scenario.add_observable(output_name)
scenario.execute(algo_name="DiagonalDOE", n_samples=20)

## Scalable diagonal discipline

### Build the scalable discipline

The second step is to build a [DataDrivenScalableDiscipline][gemseo.problems.mdo.scalable.data_driven.discipline.DataDrivenScalableDiscipline],
using a [ScalableDiagonalModel][gemseo.problems.mdo.scalable.data_driven.diagonal.ScalableDiagonalModel] and the database
converted to a [Dataset][gemseo.datasets.dataset.Dataset].



In [ ]:
dataset = scenario.to_dataset(opt_naming=False)
scalable = create_scalable("ScalableDiagonalModel", dataset)

### Visualize the input-output dependencies

We can easily access the underlying [ScalableDiagonalModel][gemseo.problems.mdo.scalable.data_driven.diagonal.ScalableDiagonalModel]
and plot the corresponding input-output dependency matrix
where the level of gray and the number (in [0,100]) represent
the degree of dependency between inputs and outputs.
Input are on the left while outputs are at the top.
More precisely, for a given output component located at the top of the graph,
these degrees are contributions to the output component and they add up to 1.
In other words, a degree expresses this contribution in percentage
and for a given column, the elements add up to 100.



In [ ]:
scalable.scalable_model.plot_dependency(save=False, show=True)

### Visualize the 1D interpolations

For every output, we can also visualize a spline interpolation of the output
samples over the diagonal of the input space.



In [ ]:
scalable.scalable_model.plot_1d_interpolations(save=False, show=True)

## Increased problem dimension

We can repeat the construction of the scalable discipline for different sizes
of variables and visualize the input-output dependency matrices.



### Twice as many inputs

For example, we can increase the size of each input by a factor of 2.



In [ ]:
sizes = {name: dataset.variable_names_to_n_components[name] * 2 for name in input_names}
scalable = create_scalable("ScalableDiagonalModel", dataset, sizes)
scalable.scalable_model.plot_dependency(save=False, show=True)

### Twice as many outputs

Or we can increase the size of each output by a factor of 2.



In [ ]:
sizes = {
    name: discipline.cache.names_to_sizes[name] * 2
    for name in discipline.io.output_grammar.names
}
scalable = create_scalable("ScalableDiagonalModel", dataset, sizes)
scalable.scalable_model.plot_dependency(save=False, show=True)

### Twice as many variables

Or we can increase the size of each input and each output by a factor of 2.



In [ ]:
names = input_names + list(discipline.io.output_grammar.names)
sizes = {name: dataset.variable_names_to_n_components[name] * 2 for name in names}
scalable = create_scalable("ScalableDiagonalModel", dataset, sizes)
scalable.scalable_model.plot_dependency(save=False, show=True)

## Binary IO dependencies

By default, any output component depends on any input component
with a random level.
We can also consider sparser input-output dependency
by means of binary input-output dependency matrices.
For that, we have to set the value of the fill factor
which represents the part of connection between inputs and outputs.
Then, a connection is represented by a black square
while an absence of connection is presented by a white one.
When the fill factor is equal to 1, any input is connected to any output.
Conversely, when the fill factor is equal to 0,
there is not a single connection between inputs and outputs.



### Fill factor = 0.2




In [ ]:
scalable = create_scalable("ScalableDiagonalModel", dataset, sizes, fill_factor=0.2)
scalable.scalable_model.plot_dependency(save=False, show=True)

### Fill factor = 0.5




In [ ]:
scalable = create_scalable("ScalableDiagonalModel", dataset, sizes, fill_factor=0.5)
scalable.scalable_model.plot_dependency(save=False, show=True)

### Fill factor = 0.8




In [ ]:
scalable = create_scalable("ScalableDiagonalModel", dataset, sizes, fill_factor=0.8)
scalable.scalable_model.plot_dependency(save=False, show=True)

## Heterogeneous dependencies




In [ ]:
scalable = create_scalable(
    "ScalableDiagonalModel", dataset, sizes, fill_factor={"y_2": 0.2}
)
scalable.scalable_model.plot_dependency(save=False, show=True)

## Group dependencies




In [ ]:
scalable = create_scalable(
    "ScalableDiagonalModel", dataset, sizes, group_dep={"y_2": ["x_shared"]}
)
scalable.scalable_model.plot_dependency(save=False, show=True)